In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
import torch.optim as optim
from torch.jit import script, trace
from torch.utils.data import DataLoader, TensorDataset

from tqdm import tqdm

from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score

In [4]:
os.getcwd()

'c:\\Users\\mauma\\OneDrive\\Documents\\UCLA\\CruX\\NTX Competition\\classifiers'

In [6]:
x_train = None
y_train = None

x_valid = None
y_valid = None

x_test = None
y_test = None

In [7]:
X = np.load('preprocessed_data.npy')
X = np.swapaxes(X, 1,2)
x_train = X

In [116]:
train_subset = torch.utils.data.TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
val_subset = torch.utils.data.TensorDataset(torch.Tensor(x_valid), torch.Tensor(y_valid))
test_dataset = torch.utils.data.TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test))

bsz = 50
train_loader = torch.utils.data.DataLoader(train_subset, shuffle=True, batch_size=bsz)
val_loader = torch.utils.data.DataLoader(val_subset, shuffle=False, batch_size=bsz)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=bsz)

In [9]:
samp_time = 750
n_channels = 8

In [10]:
class CNN_LSTM(nn.Module):
    def __init__(self):
        super(CNN_LSTM, self).__init__()
        self.conv1d = nn.Conv1d(in_channels=8, out_channels=40, kernel_size=20, stride=4)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.batchnorm1 = nn.BatchNorm1d(40)
        self.maxpool = nn.MaxPool1d(kernel_size=4, stride=4)
        self.lstm1 = nn.LSTM(input_size=45, hidden_size=30)
        self.dropout2 = nn.Dropout(p=0.5)
        self.batchnorm2 = nn.BatchNorm1d(40)
        self.lstm2 = nn.LSTM(input_size=30, hidden_size=10)
        self.dropout3 = nn.Dropout(p=0.5)
        self.batchnorm3 = nn.BatchNorm1d(40)
        self.flatten = nn.Flatten()
        self.dense = nn.Linear(400,2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1d(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.batchnorm1(x)
        x = self.maxpool(x)
        x, _ = self.lstm1(x)
        x = self.dropout2(x)
        x = self.batchnorm2(x)
        x, _ = self.lstm2(x)
        x = self.dropout3(x)
        x = self.batchnorm3(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.softmax(x)

        return x

In [117]:
input_size = (n_channels, samp_time)

# Random input tensor with the specified dimensions
input_tensor = torch.randn(1, *input_size)
print(input_tensor.shape)

# Forward pass through the model
#output_tensor = model(input_tensor)

net = CNN_LSTM()
print(net.forward(Variable(input_tensor)))
criterion = nn.CrossEntropyLoss() # add to device here
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-7)

torch.Size([1, 8, 750])
tensor([[0.4217, 0.5783]], grad_fn=<SoftmaxBackward0>)


In [119]:
y1

array([1, 1, 1, ..., 0, 0, 0])

In [118]:
train_hist = []
val_hist = []

num_epochs = 100

for epoch_idx in tqdm(range(num_epochs)):
    net.train()
    train_count = 0
    train_correct_count = 0
    for batch_idx, (train_x, train_y) in enumerate(train_loader):
        train_x = train_x.float()
        train_y = train_y.long()
        optimizer.zero_grad()
        logits = net(train_x)
        loss = criterion(logits, train_y)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            y_hat = torch.argmax(logits, dim=-1)
            train_correct_count += torch.sum(y_hat == train_y, axis=-1)
            train_count += train_x.size(0)

    train_acc = train_correct_count / train_count
    train_hist.append(train_acc)

    net.eval()
    val_count = 0
    val_correct_count = 0
    with torch.no_grad():
        for idx, (val_x, val_y) in enumerate(val_loader):
            val_x = val_x.float()
            val_y = val_y.long()
            logits = net(val_x).detach()
            y_hat = torch.argmax(logits, dim=-1)
            val_correct_count += torch.sum(y_hat == val_y, axis=-1)
            val_count += val_x.size(0)
    val_acc = val_correct_count / val_count
    val_hist.append(val_acc)

    print('Train acc: {:.3f}, Val acc: {:.3f}'.format(train_acc, val_acc))

  1%|          | 1/100 [00:01<01:40,  1.02s/it]

Train acc: 0.685, Val acc: 0.784


  2%|▏         | 2/100 [00:02<01:44,  1.07s/it]

Train acc: 0.825, Val acc: 0.844


  3%|▎         | 3/100 [00:03<01:46,  1.10s/it]

...

Train acc: 1.000, Val acc: 1.000


 99%|█████████▉| 99/100 [02:00<00:01,  1.31s/it]

Train acc: 1.000, Val acc: 1.000


100%|██████████| 100/100 [02:01<00:00,  1.22s/it]

Train acc: 1.000, Val acc: 1.000


In [120]:
net.eval()
test_count = 0
test_correct_count = 0
with torch.no_grad():
    for idx, (test_x, test_y) in enumerate(test_loader):
        test_x = test_x.float()
        test_y = test_y.long()
        logits = net(test_x).detach()
        y_hat = torch.argmax(logits, dim=-1)
        test_correct_count += torch.sum(y_hat == test_y, axis=-1)
        test_count += test_x.size(0)
test_acc = test_correct_count / test_count

print('Test acc: {:.10f}'.format(test_acc))

Test acc: 1.0000000000


### Data

In [62]:
eeg_s1 = np.load('preprocessed_data copy.npy')

In [63]:
eeg_s1.shape

(23, 8, 211, 750)

In [83]:
eeg_data = []

for i in range(23):
    j = 0
    while (j < 211 and np.sum(eeg_s1[i,:,j,:])):
        eeg_data.append(eeg_s1[i,:,j,:])
        j += 1

In [86]:
eeg_data = np.array(eeg_data)

In [90]:
indices = np.arange(1676)
np.random.seed(1)
np.random.shuffle(indices)
indices

array([ 979, 1336,  881, ..., 1096,  235, 1061])

In [92]:
test_indices = indices[1341:]
train_indices = indices[0:670]
valid_indices = indices[670:1341]

In [107]:
y1 = np.array(y1)
y2 = np.array(y2)
y3 = np.array(y3)

In [115]:
x_train = eeg_data[train_indices]
y_train = y1[train_indices]

x_valid = eeg_data[valid_indices]
y_valid = y1[valid_indices]

x_test = eeg_data[test_indices]
y_test = y1[test_indices]

### Labels

In [47]:
emocsv = pd.read_csv("emos3.csv")
emocsv2 = emocsv.iloc[:,5:]

In [48]:
emocsv2

,How did this video make you feel?,How positive were your emotions while watching the video(s)?,How energized did you feel while watching the video(s)?,"How ""dominant"" did you feel while watching the video(s)?",Content,Amused,Angry,Sad,Disgusted,Afraid,...,How positive were your emotions while watching the video(s)?.22,How energized did you feel while watching the video(s)?.22,"How ""dominant"" did you feel while watching the video(s)?.22",Content.22,Amused.22,Angry.22,Sad.22,Disgusted.22,Afraid.22,"If you had to choose one of the seven words below to best represent how you felt during the video, which one would it be?.22"
0,"euphoric, full, happy, excited",9,9,7,5,2,1,1,1,1,...,3,3,2,1,1,1,1,2,3,Afraid


In [49]:
new_cols = ["How feel", "Pos", "Energ", "Dom", "Content", "Amused", "Angry", "Sad", "Disgust", "Afraid", "Emo"]

In [51]:
emo_labels = pd.DataFrame()

for row in range(1):
    sel_row = emocsv2.iloc[row]
    for i in np.arange(0, emocsv2.shape[1], 11):
        obs = sel_row[i:i+11].to_frame().T
        obs = obs.rename(columns={obs.columns[i]: new_cols[i] for i in range(11)})
        emo_labels = pd.concat([emo_labels, obs])

In [52]:
emo_labels['Pos2'] = (emo_labels['Pos'] > 4) + 0
emo_labels['Energ2'] = (emo_labels['Energ'] > 4) + 0
emo_labels['Dom2'] = (emo_labels['Dom'] > 4) + 0

In [69]:
valence = emo_labels['Pos2'].to_numpy()
arousal = emo_labels['Energ2'].to_numpy()
dominance = emo_labels['Dom2'].to_numpy()
print(valence)
print(arousal)
print(dominance)

[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 0]
[1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0]


In [82]:
y1=[] #V, labels given for each 3-second data chunk
y2=[] #A
y3=[] #D
for i in range(23):
    j = 0
    while (j < 211 and np.sum(eeg_s1[i,:,j,:])):
        y1.append(valence[i])
        y2.append(arousal[i])
        y3.append(dominance[i])
        j += 1
print(len(y1)) #this corresponds to the number of 3-second chunks across all data chunks. 1676 -> ~ 4 minutes per video on average, checks out

1676


In [54]:
emo_labels

,How feel,Pos,Energ,Dom,Content,Amused,Angry,Sad,Disgust,Afraid,Emo,Pos2,Energ2,Dom2
0,"euphoric, full, happy, excited",9,9,7,5,2,1,1,1,1,Content,1,1,1
0,"content, at peace, excited, loving, happy",8,7,6,5,2,1,1,1,1,Content,1,1,1
0,"heartfelt, happy, loving, hopeful, sad",8,7,7,5,2,1,3,1,1,Content,1,1,1
0,"pensive, calm, supported, fulfilled",7,4,5,4,1,1,2,1,1,Content,1,0,1
0,"amused, engaged, a bit bored",7,5,7,4,4,1,1,1,1,Amused,1,1,1
0,"amused, nostalgic, feeling myself, confident",7,6,7,4,3,1,1,1,1,Content,1,1,1
0,"thoughts were wandering, relaxed, hopeful",6,4,6,3,1,1,1,1,1,Content,1,0,1
0,"contemplative, pensive, intrigued, reflective,...",7,3,6,4,1,1,2,1,1,Content,1,0,1
0,"concerned, sad, empathetic, reflective",5,2,4,1,1,1,2,1,1,Sad,1,0,0
0,"empathetic, despair, sadness, catharsis",5,2,4,2,1,1,3,1,1,Sad,1,0,0


In [59]:
torch.Tensor(emo_labels['Pos2'].to_numpy())

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.])

In [42]:
formal_emotions = emo_labels['Emo']

In [43]:
formal_emotions = formal_emotions.to_numpy()

In [44]:
formal_emotions = formal_emotions.reshape((formal_emotions.size, 1))

In [45]:
from sklearn.preprocessing import OneHotEncoder
cat = OneHotEncoder()
emotions_onehot = cat.fit_transform(formal_emotions).toarray()
emotions_onehot

array([[0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
 